In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import numpy as np

In [2]:
# Create SparkSession
ss = SparkSession.builder.appName("MapReduceExample").getOrCreate()

In [3]:
# Load Data to RDD
dataRDD = ss.read.csv("card_transdata.csv", header=True, inferSchema=True).rdd

In [ ]:
conf = SparkConf().setAppName("product_revenue")
sc = SparkContext(conf=conf)
lines = sc.textFile("path/to/data.txt")
words = lines.flatMap(lambda line: line.split(" "))

In [4]:
# Map operation
data = dataRDD.map(lambda line: np.array([float(x) for x in line.split(' ')]))
words = dataRDD.flatMap(lambda line: line.split(" "))